In [1]:
from sklearn.datasets import load_boston
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
import numpy as np

In [2]:
class Decision:
    def __init__(self, feature_index, threshold):
        self.feature_index = feature_index
        self.threshold = threshold

class Node:
    def __init__(self, y, decision, left_node, right_node):
        self.value = np.mean(y)
        self.decision = decision
        self.left_node = left_node
        self.right_node = right_node

class RegressionDecisionTree:
    def __init__(self, max_depth=10, criterion='mse'):
        self.max_depth = max_depth
        self.criterion = criterion
        self.root = None
    
    def fit(self, X, y):
        self.root = self._build_tree(X, y, 0)
        
    def predict(self, X):
        return self._predict(X, self.root)
        
    def _build_tree(self, X, y, depth):
        if depth == self.max_depth or len(X) < 2 or np.var(y) == 0:
            return Node(y, None, None, None)

        decision = self._get_best_decision(X, y)
        l_X, l_y, r_X, r_y = self._split(decision, X, y)
        
        left_node = self._build_tree(l_X, l_y, depth + 1)
        right_node = self._build_tree(r_X, r_y, depth + 1)
        return Node(y, decision, left_node, right_node)
        
    def _split(self, decision, X, y):
        l_index = X[:, decision.feature_index] <  decision.threshold
        r_index = X[:, decision.feature_index] >= decision.threshold
        
        return X[l_index], y[l_index], X[r_index], y[r_index]
        
    def _get_best_decision(self, X, y):
        best_value = None
        best_decision = None
        for feature_index in range(X.shape[1]):
            sorted_index = np.argsort(X[:, feature_index])
            l_y, r_y = y[sorted_index], []
            for threshold_index in reversed(sorted_index):
                # to avoid void arrays
                if X[threshold_index, feature_index] == X[sorted_index[0], feature_index]:
                    continue
                
                r_y.append(l_y[-1])
                l_y = l_y[:-1]
                
                if self.criterion == 'mse':
                    value = -(np.var(l_y) * len(l_y) + np.var(r_y) * len(r_y))
                else:
                    raise Exception('Unknown criterion')
                    
                if best_value is None or best_value < value:
                    best_value = value
                    best_decision = Decision(feature_index, X[threshold_index, feature_index])
                    
        return best_decision
    
    def _predict(self, X, node):
        if node.decision is None:
            return np.array([node.value] * len(X))
        
        l_X, l_index, r_X, r_index = self._split(node.decision, X, np.arange(len(X)))
        y = np.zeros(len(X))
        y[l_index] = self._predict(l_X, node.left_node)
        y[r_index] = self._predict(r_X, node.right_node)
        
        return y

In [3]:
dataset = load_boston()

In [9]:
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, train_size=0.75)

In [10]:
model = RegressionDecisionTree(max_depth=5)

In [11]:
model.fit(X_train, y_train)

In [12]:
mean_squared_error(y_test, model.predict(X_test))

19.051918364607918

In [13]:
mean_squared_error(y_train, model.predict(X_train))

7.1826724510587203

Сравним с деревом из sklearn

In [18]:
model = DecisionTreeRegressor(max_depth=5)

In [19]:
model.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [20]:
mean_squared_error(y_test, model.predict(X_test))

17.583344488029745

In [21]:
mean_squared_error(y_train, model.predict(X_train))

7.0091164103417292

Результат почти одинаков